In [7]:
import pandas as pd

ms = pd.read_csv("measurement_db-Measurement.csv")
ms

,id,batchId,xRotation,yRotation,zRotation,xAcceleration,yAcceleration,zAcceleration
0,1,1,-0.026231,0.049400,0.030093,0.223878,0.124726,9.260074
1,2,1,-0.029028,-0.008522,0.020506,0.389133,0.246909,9.851700
2,3,1,0.024900,-0.001598,0.004927,0.389133,0.246909,9.851700
3,4,1,-0.020373,-0.000533,0.003595,0.355633,-0.119641,9.767951
4,5,1,0.015446,-0.006658,0.000399,0.410818,-0.033350,9.719945
...,...,...,...,...,...,...,...,...
2195,2196,22,0.004261,-0.012250,-0.236615,0.279063,0.084048,9.731909
2196,2197,22,0.004261,-0.012250,-0.236615,0.408425,0.443420,9.595368
2197,2198,22,0.004261,-0.012250,-0.236615,0.317348,0.333201,9.710373
2198,2199,22,0.004261,-0.012250,-0.236615,0.245414,0.112762,9.935597


In [8]:
bt = pd.read_csv("measurement_db-Batch.csv")
bt

,id,type
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


In [208]:
import numpy as np
data = pd.DataFrame(columns=['type', 'sensors'])
for i in set(ms['batchId'].tolist()):
    allMeasurements = ms.loc[ms['batchId'] == i]
    filtered = allMeasurements.drop('id', inplace=False, axis=1).drop('batchId', inplace=False, axis=1).to_numpy()
    #print(allMeasurements[0])
    flat = filtered.flatten()
    batchType = bt.loc[bt['id'] == i].iat[0,1]
    data.loc[len(data)] = {
    "type": batchType,
    "sensors": flat
    }
data

,type,sensors
0,1,"[-0.026231363415718, 0.0494001805782318, 0.030..."
1,1,"[0.000932078924961388, 0.0007989247678779066, ..."
2,1,"[0.0046603945083916, -0.000932078924961388, 0...."
3,1,"[0.0647129043936729, 0.0189078859984874, 0.009..."
4,1,"[-0.0145137999206781, -0.0077229393646121, -0...."
5,1,"[-0.0246335137635469, -0.0030625448562204, -0...."
6,1,"[0.0005326165119186044, -0.0213046595454216, 0..."
7,1,"[-0.0062582441605627, 0.0050598569214344, -0.0..."
8,1,"[0.0563241951167583, 0.1213034093379974, -0.06..."
9,1,"[0.0370168462395668, -0.4901403486728668, 0.13..."
